In [1]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import math

from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from common.utils import load_data, mape
from IPython.display import Image

%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)
warnings.filterwarnings("ignore") # specify to ignore warning messages

In [18]:
# Load data
data = './data/darwin.txt'
df = pd.read_csv(data, delim_whitespace=True, skiprows=2)
df.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1951,5.3,5.1,7.0,9.4,11.9,11.2,13.3,13.2,12.5,11.4,9.5,8.9
1,1952,6.7,7.1,8.3,10.3,10.3,12.5,13.0,12.6,12.1,9.9,7.7,8.4
2,1953,6.9,7.3,8.0,8.9,13.1,12.5,13.9,13.0,13.2,10.8,9.4,8.1
3,1954,4.9,7.3,7.7,7.7,10.1,12.3,12.4,11.8,11.9,10.7,9.1,6.5
4,1955,8.3,3.7,6.7,8.9,9.4,10.9,11.7,11.4,10.9,9.3,7.6,7.6


In [42]:
tmpdf = df.drop(columns="YEAR")
tmpdf = tmpdf.dropna()
# df2 = tmpdf.sum(axis=0)
tmpdf

,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,5.3,5.1,7.0,9.4,11.9,11.2,13.3,13.2,12.5,11.4,9.5,8.9
1,6.7,7.1,8.3,10.3,10.3,12.5,13.0,12.6,12.1,9.9,7.7,8.4
2,6.9,7.3,8.0,8.9,13.1,12.5,13.9,13.0,13.2,10.8,9.4,8.1
3,4.9,7.3,7.7,7.7,10.1,12.3,12.4,11.8,11.9,10.7,9.1,6.5
4,8.3,3.7,6.7,8.9,9.4,10.9,11.7,11.4,10.9,9.3,7.6,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...
232,-0.6,0.2,-0.8,-0.2,0.2,0.5,-0.2,-0.3,0.1,-1.2,-1.4,-0.1
233,-1.7,0.2,-2.0,-0.4,0.6,0.7,-0.6,-0.3,0.1,0.2,0.5,-1.1
234,0.1,2.0,0.1,0.2,0.7,0.8,0.7,0.8,2.1,0.6,0.7,0.9
235,0.5,0.5,0.7,0.6,0.8,-0.0,-0.7,-0.8,-0.6,-0.7,0.1,-0.8


In [45]:
tmpdf.info()
for column in tmpdf.columns:
    print(tmpdf[column])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 0 to 236
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   JAN     215 non-null    object
 1   FEB     215 non-null    object
 2   MAR     215 non-null    object
 3   APR     215 non-null    object
 4   MAY     215 non-null    object
 5   JUN     215 non-null    object
 6   JUL     215 non-null    object
 7   AUG     215 non-null    object
 8   SEP     215 non-null    object
 9   OCT     215 non-null    object
 10  NOV     215 non-null    object
 11  DEC     215 non-null    object
dtypes: object(12)
memory usage: 21.8+ KB
0       5.3
1       6.7
2       6.9
3       4.9
4       8.3
       ... 
232    -0.6
233    -1.7
234     0.1
235     0.5
236    -1.0
Name: JAN, Length: 215, dtype: object
0       5.1
1       7.1
2       7.3
3       7.3
4       3.7
       ... 
232     0.2
233     0.2
234     2.0
235     0.5
236    -1.7
Name: FEB, Length: 215, dtype: object
0       7